# System Modeling

In this example we will see how to model an end-to-end optical system using prysm. Our system will have both an objective lens or telescope as well as a sensor with an optical low-pass filter. We begin by importing the relevant classes and setting some visual styles:

In [ ]:
from prysm import FringeZernike, PSF, MTF, PixelAperture, OLPF
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('bmh')

Next we model the PSF of the objective, given its aperture, focal length, and Zernike coefficients for its wavefront, such as from a Shack-Hartmann sensor or interferometer:

In [ ]:
# data from a wavefront sensor, optical design program, etc...
coefficients = [0, 0, 0, 0, 0.1, 0.1, -0.025, -0.025, 0.1]

# a circular aperture inscribed in a square 10mm on a side with 50mm EFL
# note the default mask is a circle, so the kwarg is somewhat redundant here.
pupil = FringeZernike(coefficients, dia=10, mask='circle', opd_unit='um', norm=True)
psf = PSF.from_pupil(pupil, efl=40)  # F/2

Here we have implicitly accepted the default wavelength of 0.5 microns, and Q factor of 2 (Nyquist sampling) which are usually sane defaults. The pupil is circular and is sufficiently described by a Zernike expansion up to Z9.

We can plot the wavefront or PSF of the objective.  The wavefront will appear to not quite fill the array, but this is just an artifact of the default lanczos interpolation and relatively few samples.

In [ ]:
fig, ax = pupil.plot2d(interp_method='nearest')
ax.grid(False)
ax.set_title('Wavefront')

fig, ax = psf.plot2d(axlim=20, power=2)  # 1/2 stretch, colorbar scales as well.
ax.grid(False)
ax.set_title('PSF');

or compute its MTF.  Note that "tan" and "sag" here accept the assumption of optical design code that we are looking at an object extended in Y, with no extent in X.  For example, this means we could be at an (x,y) field point of (0, 1) degrees.  On-axis, tan and sag are simply misgnomers for the "x" and "y" MTFs.

In [ ]:
mtf = MTF.from_psf(psf)
mtf.plot_tan_sag(max_freq=200)

In [ ]:
pixel_pitch = 5  # 5 micron diameter pixels
aa_filter = OLPF(pixel_pitch*0.66)
pixel = PixelAperture(pixel_pitch)
sys_psf = psf.conv(aa_filter).conv(pixel).renorm()  # renorm so max=1

We can plot the system PSF, which is abstract since it includes the pixel aperture.  You would not normally look at this, but prysm doesn't stop you from doing that.

In [ ]:
sys_psf.show(xlim=20, interp_method='lanczos', power=2)  # sys_psf is a Convolvable, not a PSF.
plt.grid(False)

In [ ]:
sys_mtf = MTF.from_psf(sys_psf)
sys_mtf.plot_tan_sag(max_freq=200)

We see the system MTF reach zero at 200 cy/mm, as would be expected for a 5 micron pixel.  We also see the PSF is significantly squared off, since the pixel aperture contribution is larger than that of the optical system.  One might find the azimuthally averaged MTF to be more informative:

In [ ]:
sys_mtf.plot_azimuthal_average(max_freq=200)

For more information on the classes used, see [Zernikes](../user_guide/Zernikes.ipynb), [PSFs](../user_guide/PSFs.ipynb), [MTFs](../user_guide/MTFs.ipynb), and [PixelApertures, OLPFs, and convolutions](../user_guide/Convolvables.ipynb)